In [ ]:
from pyspark.sql import SparkSession

# Configuração do Spark
spark = SparkSession.builder \
    .appName("Conectar ao DB2 e escrever em CSV") \
    .config("spark.driver.extraClassPath", "/path/to/db2-jdbc-driver.jar") \  # Substitua pelo caminho do driver DB2 JDBC
    .getOrCreate()

# Configuração das credenciais do banco de dados DB2
db2_url = "jdbc:db2://<hostname>:<port>/<database>"
db2_properties = {
    "user": "<seu-usuario>",
    "password": "<sua-senha>",
    "driver": "com.ibm.db2.jcc.DB2Driver"
}

# Tabela no DB2 que você deseja ler
table_name = "sua_tabela"

# Lê os dados do DB2
df = spark.read \
    .jdbc(url=db2_url, table=table_name, properties=db2_properties)

# Redistribui os dados em vários clusters
# O número de partições (clusters) depende do tamanho dos seus dados e dos recursos disponíveis
num_partitions = 4  # Você pode ajustar isso conforme necessário
df = df.repartition(num_partitions)

# Escreve os dados em um arquivo CSV
output_csv_path = "caminho/para/seu/arquivo.csv"

# Coalesce para um único cluster antes de escrever no arquivo CSV
df.coalesce(1).write \
    .csv(output_csv_path, header=True, mode="overwrite")

# Encerra a sessão do Spark
spark.stop()
